In [ ]:
import pandas as pd
import torch
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from deepctr_torch.inputs import SparseFeat, get_feature_names, DenseFeat
from deepctr_torch.models import DeepFM
import deepctr_torch
from sklearn.metrics import log_loss
import numpy as np
import gc
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, MinMaxScaler
from tqdm import tqdm

In [ ]:

train = pd.read_pickle(f'../temp_data/df_train_v6.pickle')
val = pd.read_pickle('../temp_data/df_valid_v6.pickle')
test = pd.read_pickle('../temp_data/df_test_v6.pickle')

In [ ]:
ycol = 'is_finish'

feats = ['userid','videoid','tag', 'videoid_cnt', 'videoid_is_like_sum',
       'videoid_is_favourite_sum', 'videoid_is_share_sum',
       'videoid_is_finish_sum', 'videoid_is_like_mean',
       'videoid_is_favourite_mean', 'videoid_is_share_mean',
       'videoid_is_finish_mean', 'userid_cnt', 'userid_is_like_sum',
       'userid_is_favourite_sum', 'userid_is_share_sum',
       'userid_is_finish_sum', 'userid_is_like_mean',
       'userid_is_favourite_mean', 'userid_is_share_mean',
       'userid_is_finish_mean']
train = train[feats+['is_finish']]
val = val[feats+['is_finish']]
test = test[feats]

In [ ]:
sparse_features = ['userid','videoid','tag']#'userid',
dense_features = [i for i in train.columns if i not in ['is_finish','tag','userid','videoid','flag','is_like', 'is_share', 'is_favourite']]
train[dense_features] = train[dense_features].fillna(0.0, )
val[dense_features] = val[dense_features].fillna(0.0, )
test[dense_features] = test[dense_features].fillna(0.0, )

In [ ]:
for i in tqdm(dense_features):
    train[i] = (train[i] - train[i].min())/(train[i].max() - train[i].min())
    val[i] = (val[i] - val[i].min())/(val[i].max() - val[i].min())
    test[i] = (test[i] - test[i].min())/(test[i].max() - test[i].min())
fixlen_feature_columns = [SparseFeat('userid',  vocabulary_size=500000, embedding_dim=8),
                         SparseFeat('videoid', vocabulary_size=500000, embedding_dim=8),
                         SparseFeat('tag', vocabulary_size=train['tag'].nunique(), embedding_dim=8)] + [DenseFeat(feat, 1, )for feat in dense_features]
dnn_feature_columns = fixlen_feature_columns
linear_feature_columns = fixlen_feature_columns
feature_names = get_feature_names(linear_feature_columns + dnn_feature_columns)
train_model_input = {name: train[name] for name in feature_names}
val_model_input = {name: val[name] for name in feature_names}
test_model_input = {name: test[name] for name in feature_names}

In [ ]:
from tqdm import tqdm
from torch.optim import Adagrad
model = deepctr_torch.models.wdl.WDL(linear_feature_columns=linear_feature_columns, dnn_feature_columns=dnn_feature_columns,
                   task='binary',
                   l2_reg_embedding=1e-5, device='cpu')#,dnn_hidden_units=(256, 256, 256)
model.compile(Adagrad(model.parameters(),0.1), "binary_crossentropy",
              metrics=["binary_crossentropy", "auc"], )
for epoch in tqdm(range(2)):
    model.fit(train_model_input,train['is_finish'].values,batch_size=1024,epochs=1,verbose=2)
    pred = model.predict(val_model_input, 4096)
    print('epoch ', epoch, 'log_loss: ', log_loss(val['is_finish'],pred))
    

predict = model.predict(test_model_input, 4096)

In [ ]:
sub = pd.read_csv('../init_data/toUser/test/test.csv')
sub['is_finish'] = predict
sub[['ID','is_finish']].to_csv('../temp_data/nn.csv', index=None)